In [1]:
!pip install kaggle


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
from zipfile import ZipFile

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [4]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [5]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [6]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()   # unzip the dataset file

In [8]:
#Loading the dataset
data = pd.read_csv("IMDB Dataset.csv")

In [9]:
data.shape

(50000, 2)

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [12]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [13]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\tanur\AppData\Local\Temp\ipykernel_3448\2568826810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [14]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [15]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [17]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

Y_train = np.array(train_data["sentiment"])
Y_test = np.array(test_data["sentiment"])

In [18]:
# Build CNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(Conv1D(filters=128, kernel_size=5, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

C:\Users\tanur\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [21]:
# Train the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 118s 209ms/step - accuracy: 0.6079 - loss: 0.6217 - val_accuracy: 0.8639 - val_loss: 0.3195
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 103s 206ms/step - accuracy: 0.8749 - loss: 0.3038 - val_accuracy: 0.8863 - val_loss: 0.2745
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 102s 203ms/step - accuracy: 0.9232 - loss: 0.2071 - val_accuracy: 0.8838 - val_loss: 0.2804
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 200ms/step - accuracy: 0.9457 - loss: 0.1502 - val_accuracy: 0.8816 - val_loss: 0.3090
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 198ms/step - accuracy: 0.9636 - loss: 0.1075 - val_accuracy: 0.8776 - val_loss: 0.3541


In [22]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.8804 - loss: 0.3301
Test Loss: 0.32430538535118103
Test Accuracy: 0.8809000253677368


In [23]:
# Define prediction function
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

In [24]:
# Example predictions
reviews = [
    "The movie was fantastic! I loved it.",
    "The movie was terrible. I hated it.",
    "It was okay, not great but not bad."
]

for review in reviews:
    sentiment = predict_sentiment(review)
    print(f"Review: \"{review}\" | Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
Review: "The movie was fantastic! I loved it." | Sentiment: positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Review: "The movie was terrible. I hated it." | Sentiment: negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
Review: "It was okay, not great but not bad." | Sentiment: negative
